<a href="https://colab.research.google.com/github/cflores23/optimizacion/blob/main/Polynomial_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sympy as sp #para variables simbólicas
import sys

In [ ]:
df = pd.read_csv('coronavirusglobal.csv')
df.head()

,T,Date,Daily Cases,Daily Death,Daily Recovered,Total Cases,Total Death,Total Recovered,Active Cases
0,1,21-Jan-20,60,3,0,282,3,0,279
1,2,22-Jan-20,32,3,0,314,6,0,308
2,3,23-Jan-20,265,8,34,579,14,34,531
3,4,24-Jan-20,472,16,4,1051,30,38,983
4,5,25-Jan-20,698,15,11,1749,45,49,1655


In [ ]:
df = pd.read_csv('coronavirusglobal.csv', usecols=['Daily Cases','Daily Death'])
df.head()

,Daily Cases,Daily Death
0,60,3
1,32,3
2,265,8
3,472,16
4,698,15


In [ ]:

#Construir matriz del sistema de ecuaciones
def const_matriz(x,y,n,n_reg):
  a = np.zeros((n+1,n+2))
  b = np.empty((n+1, 0), int)
  tam_a=0
  print(a)
  a[0][0] = n_reg

  sum_x = np.sum(x, axis=0)
  sum_y = np.sum(y,axis=0)
  sum_xy=0
  sumx_cuad = 0
  sumx_cuady = 0
  sum_x_xcub =0
  sum_x_exp4 = 0
  vec_sumatorias = np.zeros(n+2)
  #vec_sumatorias_y = np.zeros()
  vec_sumatorias_y = np.zeros(n+1)
  #Ingresar la sumatoria de y al array de y
  vec_sumatorias_y[0] = sum_y
  for i in range(n_reg):
    sum_xy += x[i]*y[i]
    sumx_cuad += x[i]*x[i]
    sumx_cuady += (x[i]*x[i])*y[i]
    sum_x_xcub += x[i]*x[i]*x[i]
    sum_x_exp4 += x[i]*x[i]*x[i]*x[i]
  
    vec_sumatorias[0] = sum_x
    vec_sumatorias[1] = sumx_cuad
    vec_sumatorias[2] = sum_x_xcub
    vec_sumatorias[3] = sum_x_exp4

  for i in range(n):
    a[0][i+1] = vec_sumatorias[i]
  for i in range(n):
    for j in range(n+1):
      a[i+1][j] = vec_sumatorias[j+i]
  #Agregar el la sumatoria de x*y al array 
  vec_sumatorias_y[1] = sum_xy
  for i in range(vec_sumatorias_y.size-2):
    vec_sumatorias_y[i+2] = sumx_cuady


  #Agregar el vector sumatorias de y a la matriz extendida
  tam_a = a.shape[1]
  for i in range(vec_sumatorias_y.size):
    a[i][tam_a-1] = vec_sumatorias_y[i]
  

  #Gauss Jordan
  print('Matriz aumentada: ')
  print(a)

  filas_a = a.shape[0]

  # Para cada fila en AB
  for i in range(0,filas_a-1,1):
    # columna desde diagonal i en adelante
    columna = abs(a[i:,i])
    dondemax = np.argmax(columna)
    
    # dondemax no está en diagonal
    if (dondemax !=0):
        # intercambia filas
        temporal = np.copy(a[i,:])
        a[i,:] = a[dondemax+i,:]
        a[dondemax+i,:] = temporal
        
  a1 = np.copy(a)
  #print('Pivoteo parcial por filas')
  #print(a1)

  # eliminacion hacia adelante
  for i in range(0,filas_a-1,1):
    pivote = a[i,i]
    #print('pivote'+str(pivote))
    adelante = i + 1
    for k in range(adelante,filas_a,1):
        factor = a[k,i]/pivote
        #print('factor'+str(factor))
        a[k,:] = a[k,:] - a[i,:]*factor
  a2 = np.copy(a)
  #print('eliminacion hacia adelante')
  #print(a2)

  # elimina hacia atras
  ultfila = filas_a-1
  ultcolumna = tam_a-1
  for i in range(ultfila,0-1,-1):
    pivote = a[i,i]
    atras = i-1 
    for k in range(atras,0-1,-1):
        factor = a[k,i]/pivote
        a[k,:] = a[k,:] - a[i,:]*factor
    # diagonal a unos
    a[i,:] = a[i,:]/a[i,i]
  X = np.copy(a[:,ultcolumna])
  X = np.transpose([X])

  print('Vector solución: ')
  print(X)

In [ ]:
x = df['Daily Cases'].tolist()
y = df['Daily Death'].tolist()
#z = df['Daily Recovered'].tolist()
n_var = len(df.axes[1])
rows = len(df.axes[0]) 
print(n_var)
print(rows)
mat = const_matriz(x,y,n_var,rows)

2
142
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Matriz aumentada: 
[[1.42000000e+02 7.44416900e+06 6.67702558e+11 4.16979000e+05]
 [7.44416900e+06 6.67702558e+11 6.46239289e+16 3.61550558e+10]
 [6.67702558e+11 6.46239289e+16 6.54555563e+21 3.34041355e+15]]
Vector solución: 
[[-3.02253373e+02]
 [ 1.15685569e-01]
 [-6.00992079e-07]]
